In [7]:
import numpy as np
import json
import pickle
from argparse import ArgumentParser, Namespace
from pathlib import Path
from typing import Dict
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# from tqdm import trange

from dataset import SeqClsDataset, SeqTaggingClsDataset
from utils import Vocab
from sklearn.metrics import accuracy_score

from importlib import reload
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score
from seqeval.metrics import precision_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import recall_score

In [8]:
TRAIN = "train"
DEV = "eval"
SPLITS = [TRAIN, DEV]
cache_dir = Path("cache/slot/")
data_dir = Path("data/slot/")
with open("cache/slot/vocab.pkl", "rb") as f:
        vocab: Vocab = pickle.load(f)
slot_idx_path = Path("cache/slot/tag2idx.json")
slot2idx: Dict[str, int] = json.loads(slot_idx_path.read_text()) ##dict of classes
data_paths = {split: data_dir / f"{split}.json" for split in SPLITS} ## 2 items, a dict
data = {split: json.loads(path.read_text()) for split, path in data_paths.items()}
datasets: Dict[str, SeqTaggingClsDataset] = {
    split: SeqTaggingClsDataset(split_data, vocab, slot2idx, 128)
    for split, split_data in data.items()
}
data = json.loads(Path("./data/slot/test.json").read_text())
dataset = SeqTaggingClsDataset(data, vocab, slot2idx, 128, test=True)

In [9]:
y_true = [item['tags'] for item in datasets["eval"]]
y_pred = pd.read_csv("eval.csv")['tags'].to_list()
y_pred = [item.split(" ") for item in y_pred]
print("accuary: ", accuracy_score(y_true, y_pred))
print("p: ", precision_score(y_true, y_pred))
print("r: ", recall_score(y_true, y_pred))
print("f1: ", f1_score(y_true, y_pred))
print(classification_report(y_true, y_pred, mode='strict', scheme=IOB2))

accuary:  0.9680648840451147
p:  0.792147806004619
r:  0.8147268408551069
f1:  0.8032786885245903
              precision    recall  f1-score   support

        date       0.72      0.74      0.73       206
  first_name       0.99      0.98      0.99       102
   last_name       0.92      0.94      0.93        78
      people       0.71      0.72      0.72       238
        time       0.83      0.87      0.85       218

   micro avg       0.80      0.81      0.81       842
   macro avg       0.83      0.85      0.84       842
weighted avg       0.80      0.81      0.81       842

